In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import cv2
import tensorflow as tf
import os
import shutil
import csv
from PIL import Image

In [2]:
N1 = 69225 % 2

### Calculando a área da Celula

In [17]:
def calculaAreaPerimetroImagem(img_cv2):
  
  imagem_cinza = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2GRAY)
  _, mascara_binaria = cv2.threshold(imagem_cinza, 1, 255, cv2.THRESH_BINARY)
  contornos, _ = cv2.findContours(mascara_binaria, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  mascara_colorida = np.zeros_like(img_cv2)
  cv2.drawContours(mascara_colorida, contornos, -1, (255, 255, 255), thickness=cv2.FILLED)

  # Exibir a imagem original, a máscara e a parte colorida
  area_branca = np.sum(mascara_binaria == 255)
  
  
  total_perimetro = 0
  
  for contorno in contornos:
    perimeter = cv2.arcLength(contorno, closed=True)
    total_perimetro += perimeter
  
  return area_branca, total_perimetro


In [18]:
def calcularCompacidade(img_cv2):
  area, perimetro = calculaAreaPerimetroImagem(img_cv2)
  
  compacidade = area / (perimetro**2)
  
  return compacidade

In [16]:
def calcularExcentricidades(img_cv2):
  
  imagem_cinza = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2GRAY)
  _, mascara_binaria = cv2.threshold(imagem_cinza, 1, 255, cv2.THRESH_BINARY)
  contornos, _ = cv2.findContours(mascara_binaria, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  maior_contorno = max(contornos, key=cv2.contourArea)

    
  if len(maior_contorno) >= 5:
    (elipse_x, elipse_y), (eixo_maior, eixo_menor), _ = cv2.fitEllipse(maior_contorno)
    # Restante do seu código
  else:
    return "-"


  # Calcular excentricidade
  
  if(eixo_maior != 0 and eixo_menor != 0):
    value_mid = round((eixo_menor / eixo_maior)**2, 2)
  else:
    return "-"
  
  excentricidade = round(np.sqrt(value_mid-1),2)
  
  return excentricidade

In [19]:
csv = {'id_cel': [],
       'area': [],
       'compacidade': [],
       'excenticridade': [],
       }

path_dataset_segmentation = "./data/segmentation_dataset/"


for pasta_atual, subpastas, arquivos in os.walk(path_dataset_segmentation):
    for arquivo in arquivos:
        caminho_completo = os.path.join(pasta_atual, arquivo)
        
        imagem = cv2.imread(caminho_completo)
        
        
        area, _ = calculaAreaPerimetroImagem(imagem)
        
        nomeSplit = caminho_completo.split("\\")
        nomeCel = nomeSplit[1].replace(".png", "")
        
        csv['id_cel'].append(nomeCel)

        if(area > 10):
          csv['area'].append(round(area,2))
          csv['compacidade'].append(calcularCompacidade(imagem))
          csv['excenticridade'].append(calcularExcentricidades(imagem))
        else:
          csv['area'].append("-")
          csv['compacidade'].append("-")
          csv['excenticridade'].append("-")
          
  
       
          
df = pd.DataFrame(csv)
df.to_csv('csv_pt2.csv', index=False)

C:\Users\Viny\AppData\Local\Temp\ipykernel_26708\2304978238.py:4: RuntimeWarning: divide by zero encountered in divide
  compacidade = area / (perimetro**2)
